# **Best Location for a new Italian Restaurant in Ottawa**

#### 1. *Install and import dependencies*

In [9]:
# install folium 
! pip install folium==0.5.0
import folium # plotting library

# install geopy
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### *2. Set the Fousquare API identifications*

In [10]:
# set the credential to access the API
CLIENT_ID = 'JB5U4X4LBYC3DFQQQQ5APU3WSSM0NRQZLUJILG1EBBGNQEBK' # your Foursquare ID
CLIENT_SECRET = 'NMFGWO35LCH24LJHZQJUDLV1TWNILFKP1V0VPSCK5VGAFBAE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

# set the address for the inquired city
address = 'Ottawa, ON, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# search query and search radius
search_query = 'Italian'
radius = 100000

# set the url identifier for the Forsquare API
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# set the result of the request
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet
0,4fb02dd4e4b03338fb817794,Italian Party Central,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",v-1606090698,False,45.419770,-75.700840,"[{'label': 'display', 'lat': 45.41977, 'lng': ...",836,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN
1,538184ec498ef9ed050db5fc,il mercato Italian Eatery,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1606090698,False,45.427519,-75.692808,"[{'label': 'display', 'lat': 45.42751875566308...",740,CA,Canada,"[55 Byward Market Sq, Ottawa ON K1N 9C3, Canada]",55 Byward Market Sq,K1N 9C3,Ottawa,ON,NaN
2,4c23d036136d20a1aed0e061,Robbie's Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1606090698,False,45.411511,-75.630156,"[{'label': 'display', 'lat': 45.411511, 'lng':...",4820,CA,Canada,"[1531 St. Laurent Blvd (Belfast Rd.), Ottawa O...",1531 St. Laurent Blvd,K1G 1A1,Ottawa,ON,Belfast Rd.
3,4fdcf585e4b0a5a27f81fec7,Italian Week,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",v-1606090698,False,45.405094,-75.709014,"[{'label': 'display', 'lat': 45.40509376252331...",2305,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN
4,4c3f3bc8da3dc9289e8fc5b9,La Strada Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1606090698,False,45.405188,-75.689707,"[{'label': 'display', 'lat': 45.40518808766283...",1772,CA,Canada,"[Bank St. (Glebe Ave.), Ottawa ON, Canada]",Bank St.,NaN,Ottawa,ON,Glebe Ave.


#### *3. Refine the data*

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,Italian Party Central,Cocktail Bar,45.419770,-75.700840,"[{'label': 'display', 'lat': 45.41977, 'lng': ...",836,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4fb02dd4e4b03338fb817794
1,il mercato Italian Eatery,Pizza Place,45.427519,-75.692808,"[{'label': 'display', 'lat': 45.42751875566308...",740,CA,Canada,"[55 Byward Market Sq, Ottawa ON K1N 9C3, Canada]",55 Byward Market Sq,K1N 9C3,Ottawa,ON,NaN,538184ec498ef9ed050db5fc
2,Robbie's Italian Restaurant,Italian Restaurant,45.411511,-75.630156,"[{'label': 'display', 'lat': 45.411511, 'lng':...",4820,CA,Canada,"[1531 St. Laurent Blvd (Belfast Rd.), Ottawa O...",1531 St. Laurent Blvd,K1G 1A1,Ottawa,ON,Belfast Rd.,4c23d036136d20a1aed0e061
3,Italian Week,General Entertainment,45.405094,-75.709014,"[{'label': 'display', 'lat': 45.40509376252331...",2305,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4fdcf585e4b0a5a27f81fec7
4,La Strada Italian Restaurant,Italian Restaurant,45.405188,-75.689707,"[{'label': 'display', 'lat': 45.40518808766283...",1772,CA,Canada,"[Bank St. (Glebe Ave.), Ottawa ON, Canada]",Bank St.,NaN,Ottawa,ON,Glebe Ave.,4c3f3bc8da3dc9289e8fc5b9
5,Dantessa,Italian Restaurant,45.419239,-75.689406,"[{'label': 'display', 'lat': 45.41923858769273...",219,CA,Canada,"[1675 Tenth Line Rd., Orléans ON, Canada]",1675 Tenth Line Rd.,NaN,Orléans,ON,NaN,4b9a7b08f964a52035b935e3
6,Industria Italian Brasserie,Italian Restaurant,45.400006,-75.684589,"[{'label': 'display', 'lat': 45.40000563686458...",2391,CA,Canada,"[225 Marche Way, Ottawa ON, Canada]",225 Marche Way,NaN,Ottawa,ON,NaN,55ac165a498ed9bf33c8a6f1
7,Giovanni's Fine Italian Cuisine,Italian Restaurant,45.401287,-75.710123,"[{'label': 'display', 'lat': 45.40128666469074...",2695,CA,Canada,"[362 Preston Street (in Little Italy), Ottawa ...",362 Preston Street,K1S 4M7,Ottawa,ON,in Little Italy,4e64110e2271573ad853c1f9
8,Roger's Italian Bistro,Restaurant,45.414637,-75.648327,"[{'label': 'display', 'lat': 45.414637, 'lng':...",3358,CA,Canada,"[500 Terminal Ave, Ottawa ON K1G 0Z3, Canada]",500 Terminal Ave,K1G 0Z3,Ottawa,ON,NaN,519d3a78498e440d98462367
9,Napolis Italian Restaurant,Pizza Place,45.396936,-75.743153,"[{'label': 'display', 'lat': 45.396936, 'lng':...",4928,CA,Canada,"[81 richmond rd (Island park dr), Ottawa ON, C...",81 richmond rd,NaN,Ottawa,ON,Island park dr,4bf3122c94af2d7fa9133972


In [12]:
# set the map
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the center of the search
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

#### *The map shows the restaurants are pretty diverse over the city and more dense around the downtown area*

#### *5. Visualize the location of the most dense Italians descent*

In [13]:
# add a red circle marker to represent the center of the search
address = '341 Preston St, Ottawa ON'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# add Little Italy area to the map as green circle marker
folium.CircleMarker(
    [45.4019058 , -75.7102969],
    radius=20,
    color='red',
    popup='Little Italy',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)
venues_map

#### *This map shows that the Italians are more likely to be more condence in this area*

In [14]:
# add Little Italy area to the map as green circle marker
folium.CircleMarker(
    [45.398299 , -75.709726],
    radius=5,
    color='yellow',
    popup='New',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)
venues_map

#### *The best location is set to be around the yellow area*